In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!unzip "/gdrive/My Drive/classified.zip"

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from skimage import io

In [ ]:
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    "classified/", 
    target_size=(128, 128),
    color_mode="grayscale",
    batch_size=64,
    class_mode='categorical'
)

# validation_generator = train_datagen.flow_from_directory(
#     "classified/", 
#     target_size=(128, 128),
#     color_mode="grayscale",
#     batch_size=64,
#     class_mode='categorical',
#     subset='validation'
# )


Found 13544 images belonging to 14 classes.


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dropout(.2, input_shape=(128, 128, 1)),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                              tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(14, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 14, 14, 64)      

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    verbose=1
)

Epoch 1/10
211/211 [==============================] - 8s 37ms/step - loss: 1.4154 - accuracy: 0.5375
Epoch 2/10
211/211 [==============================] - 8s 37ms/step - loss: 0.3015 - accuracy: 0.9085
Epoch 3/10
211/211 [==============================] - 8s 37ms/step - loss: 0.1675 - accuracy: 0.9501
Epoch 4/10
211/211 [==============================] - 8s 37ms/step - loss: 0.1125 - accuracy: 0.9655
Epoch 5/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0940 - accuracy: 0.9742
Epoch 6/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0751 - accuracy: 0.9792
Epoch 7/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0628 - accuracy: 0.9826
Epoch 8/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0570 - accuracy: 0.9832
Epoch 9/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0531 - accuracy: 0.9845
Epoch 10/10
211/211 [==============================] - 8s 37ms/step - loss: 0.0426 - accura

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
    verbose=1
)

Epoch 1/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0313 - accuracy: 0.9886 - val_loss: 0.0335 - val_accuracy: 0.9911
Epoch 2/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0317 - accuracy: 0.9886 - val_loss: 0.0496 - val_accuracy: 0.9888
Epoch 3/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0330 - accuracy: 0.9885 - val_loss: 0.0587 - val_accuracy: 0.9844
Epoch 4/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0330 - accuracy: 0.9889 - val_loss: 0.0598 - val_accuracy: 0.9918
Epoch 5/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0324 - accuracy: 0.9881 - val_loss: 0.0347 - val_accuracy: 0.9896
Epoch 6/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0324 - accuracy: 0.9876 - val_loss: 0.0460 - val_accuracy: 0.9926
Epoch 7/10
211/211 [==============================] - 8s 39ms/step - loss: 0.0291 - accuracy: 0.9883 - val_loss: 0.0343 - val_accuracy: 0.9926

In [ ]:
model.save('recognizer/')

INFO:tensorflow:Assets written to: recognizer/assets


In [ ]:
! zip -r recognizer.zip recognizer

  adding: recognizer/ (stored 0%)
  adding: recognizer/keras_metadata.pb (deflated 93%)
  adding: recognizer/variables/ (stored 0%)
  adding: recognizer/variables/variables.index (deflated 70%)
  adding: recognizer/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: recognizer/saved_model.pb (deflated 89%)
  adding: recognizer/assets/ (stored 0%)


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import argparse
import csv

dic = {1: 8, 2: 5, 3: 4, 5: 9, 6: 1, 8: 7, 9: 6, 10: 3, 12: 2, 13: 0}
operators = [0, 4, 7, 11]


def calculate(array):
    cnt = 0

    numbers = []
    to_do = 0


    for i in array:
        if i in operators:
            cnt += 1
            to_do = i
        else:
            numbers.append(dic[i])

    if (cnt != 1): return 0
    try:
        if (to_do == 0): return numbers[0] / numbers[1]
        if (to_do == 4): return numbers[0] - numbers[1]
        if (to_do == 7): return numbers[0] + numbers[1]
        if (to_do == 11): return numbers[0] * numbers[1]
    except:
        return 0


parser = argparse.ArgumentParser()
parser.add_argument("folder", type=str)
args = parser.parse_args()

folder_path = args.folder # image folder

# folder_path = "test/"
model_path = 'recognizer/' # model folder

img_width, img_height = 384, 128
M, N = 128, 128

model = load_model(model_path) # load the trained model

images = []
files = os.listdir(folder_path)
for img in files:
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_height, img_width),color_mode='grayscale')
    img = image.img_to_array(img)
    img /= 255.
    tiles = [img[x:x+M,y:y+N] for x in range(0,img.shape[0],M) for y in range(0,img.shape[1],N)]

    img = np.expand_dims(tiles[0], axis=0)
    images.append(img)
    img = np.expand_dims(tiles[1], axis=0)
    images.append(img)
    img = np.expand_dims(tiles[2], axis=0)
    images.append(img)

images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
k = list(classes)
answers = []
for i in range(len(files)):
  p = k[3*i: 3*i + 3]
  ans = 0
  try:
    ans = calculate(p)
  except:
    ans = 0
  answers.append([files[i], ans])

with open("team_name_2.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(answers)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[['42.jpg', 2], ['63.jpg', 63], ['41.jpg', 7], ['43.jpg', 18], ['64.jpg', 2]]
